In [18]:
#Question1
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, hour, date_format, concat_ws, to_timestamp, count, row_number
)
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("HW2-Q1").getOrCreate()


ConnectionRefusedError: [Errno 111] Connection refused

In [26]:
# read CSV file
bakery = spark.read.csv("shared/hw2/Bakery.csv", header=True, inferSchema=True)

# check
bakery.show(5)
bakery.printSchema()



+----------+-------------------+-----------+-------------+
|      Date|               Time|Transaction|         Item|
+----------+-------------------+-----------+-------------+
|2016-10-30|2025-03-22 09:58:11|          1|        Bread|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|
|2016-10-30|2025-03-22 10:07:57|          3|Hot chocolate|
|2016-10-30|2025-03-22 10:07:57|          3|          Jam|
+----------+-------------------+-----------+-------------+
only showing top 5 rows

root
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Transaction: integer (nullable = true)
 |-- Item: string (nullable = true)



In [27]:
bakery = bakery.withColumn(
    "DateTime",
    to_timestamp(col("Time"), "yyyy-MM-dd HH:mm:ss")
)

#check
bakery.show(5)
bakery.printSchema()


+----------+-------------------+-----------+-------------+-------------------+
|      Date|               Time|Transaction|         Item|           DateTime|
+----------+-------------------+-----------+-------------+-------------------+
|2016-10-30|2025-03-22 09:58:11|          1|        Bread|2025-03-22 09:58:11|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|
|2016-10-30|2025-03-22 10:07:57|          3|Hot chocolate|2025-03-22 10:07:57|
|2016-10-30|2025-03-22 10:07:57|          3|          Jam|2025-03-22 10:07:57|
+----------+-------------------+-----------+-------------+-------------------+
only showing top 5 rows

root
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Transaction: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)



In [28]:
filtered = bakery.filter(
    (hour(col("DateTime")) >= 6) & 
    (hour(col("DateTime")) < 10)
)

filtered.show(5)


+----------+-------------------+-----------+-------+-------------------+
|      Date|               Time|Transaction|   Item|           DateTime|
+----------+-------------------+-----------+-------+-------------------+
|2016-10-30|2025-03-22 09:58:11|          1|  Bread|2025-03-22 09:58:11|
|2016-10-31|2025-03-22 08:28:31|         81| Coffee|2025-03-22 08:28:31|
|2016-10-31|2025-03-22 08:28:31|         81|   Cake|2025-03-22 08:28:31|
|2016-10-31|2025-03-22 08:47:05|         82|Tartine|2025-03-22 08:47:05|
|2016-10-31|2025-03-22 08:47:05|         82|   NONE|2025-03-22 08:47:05|
+----------+-------------------+-----------+-------+-------------------+
only showing top 5 rows



In [17]:
# Day of extraction
filtered_with_day = filtered.withColumn(
    "Day",
    date_format(col("DateTime"), "EEEE")  
)

# Group by Day and Item and count the quantity
grouped = filtered_with_day.groupBy("Day", "Item").agg(count("*").alias("Qty"))

window_spec = Window.partitionBy("Day").orderBy(col("Qty").desc())

# rank
ranked = grouped.withColumn("rank", row_number().over(window_spec))

# top10
top10 = ranked.filter(col("rank") <= 10).drop("rank")


NameError: name 'filtered' is not defined

In [30]:
result = top10.orderBy("Day", col("Qty").desc())
final_result = top10.orderBy("Day", col("Qty").desc()).select("Item", "Day", "Qty")
final_result.show(50, truncate=False)



+-------------+--------+---+
|Item         |Day     |Qty|
+-------------+--------+---+
|Coffee       |Saturday|795|
|Bread        |Saturday|573|
|Pastry       |Saturday|250|
|Medialuna    |Saturday|169|
|Tea          |Saturday|124|
|NONE         |Saturday|104|
|Toast        |Saturday|88 |
|Farm House   |Saturday|77 |
|Hot chocolate|Saturday|65 |
|Cookies      |Saturday|51 |
+-------------+--------+---+



In [38]:
#Question2
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, hour, to_timestamp, when, collect_list,
    explode, count, row_number, concat, lit
)
from pyspark.sql.window import Window

spark2 = SparkSession.builder.appName("HW2-Q2").getOrCreate()

bakery = spark2.read.csv("shared/hw2/Bakery.csv", header=True, inferSchema=True)


bakery = bakery.withColumn(
    "DateTime",
    to_timestamp(col("Time"), "yyyy-MM-dd HH:mm:ss")
)


bakery.show(5)
bakery.printSchema()


+----------+-------------------+-----------+-------------+-------------------+
|      Date|               Time|Transaction|         Item|           DateTime|
+----------+-------------------+-----------+-------------+-------------------+
|2016-10-30|2025-03-22 09:58:11|          1|        Bread|2025-03-22 09:58:11|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|
|2016-10-30|2025-03-22 10:07:57|          3|Hot chocolate|2025-03-22 10:07:57|
|2016-10-30|2025-03-22 10:07:57|          3|          Jam|2025-03-22 10:07:57|
+----------+-------------------+-----------+-------------+-------------------+
only showing top 5 rows

root
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Transaction: integer (nullable = true)
 |-- Item: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)



In [39]:
bakery = bakery.withColumn(
    "Daypart",
    when((hour(col("DateTime")) >= 6) & (hour(col("DateTime")) <= 10), "morning")
    .when((hour(col("DateTime")) >= 11) & (hour(col("DateTime")) <= 15), "midday")
    .otherwise("evening")
)
bakery.show(5)

+----------+-------------------+-----------+-------------+-------------------+-------+
|      Date|               Time|Transaction|         Item|           DateTime|Daypart|
+----------+-------------------+-----------+-------------+-------------------+-------+
|2016-10-30|2025-03-22 09:58:11|          1|        Bread|2025-03-22 09:58:11|morning|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|morning|
|2016-10-30|2025-03-22 10:05:34|          2| Scandinavian|2025-03-22 10:05:34|morning|
|2016-10-30|2025-03-22 10:07:57|          3|Hot chocolate|2025-03-22 10:07:57|morning|
|2016-10-30|2025-03-22 10:07:57|          3|          Jam|2025-03-22 10:07:57|morning|
+----------+-------------------+-----------+-------------+-------------------+-------+
only showing top 5 rows



In [40]:
from pyspark.sql.functions import collect_list

items_by_tx = bakery.groupBy("Transaction", "Daypart") \
    .agg(collect_list("Item").alias("Items"))

items_by_tx.show(5, truncate=False)


+-----------+-------+-----------------------------+
|Transaction|Daypart|Items                        |
+-----------+-------+-----------------------------+
|1          |morning|[Bread]                      |
|2          |morning|[Scandinavian, Scandinavian] |
|3          |morning|[Hot chocolate, Jam, Cookies]|
|4          |morning|[Muffin]                     |
|5          |morning|[Coffee, Pastry, Bread]      |
+-----------+-------+-----------------------------+
only showing top 5 rows



In [41]:
items_exploded_1 = items_by_tx.select(
    "Transaction",
    "Daypart",
    explode("Items").alias("Item1")
)

items_exploded_2 = items_by_tx.select(
    "Transaction",
    "Daypart",
    explode("Items").alias("Item2")
)

pairs = items_exploded_1.join(
    items_exploded_2,
    on=["Transaction", "Daypart"],  
    how="inner"
).where(col("Item1") < col("Item2")) 
pairs.show(5, truncate=False)

pairs_count = pairs.groupBy("Daypart", "Item1", "Item2") \
    .agg(count("*").alias("Qty"))

pairs_count.show(5, truncate=False)


+-----------+-------+-------------+-------------+
|Transaction|Daypart|Item1        |Item2        |
+-----------+-------+-------------+-------------+
|3          |morning|Hot chocolate|Jam          |
|3          |morning|Cookies      |Jam          |
|3          |morning|Cookies      |Hot chocolate|
|5          |morning|Coffee       |Pastry       |
|5          |morning|Bread        |Pastry       |
+-----------+-------+-------------+-------------+
only showing top 5 rows

+-------+-----------------+-----------------+---+
|Daypart|Item1            |Item2            |Qty|
+-------+-----------------+-----------------+---+
|midday |Frittata         |Hearty & Seasonal|2  |
|midday |Hearty & Seasonal|Smoothies        |6  |
|midday |Art Tray         |Bacon            |1  |
|midday |Farm House       |Fudge            |2  |
|midday |Juice            |Vegan mincepie   |1  |
+-------+-----------------+-----------------+---+
only showing top 5 rows



In [42]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy("Daypart").orderBy(col("Qty").desc())
ranked_pairs = pairs_count.withColumn("rn", row_number().over(window_spec))

top_pairs = ranked_pairs.filter(col("rn") == 1).drop("rn")

formatted = top_pairs.select(
    concat(
        col("Daypart"),              # morning
        lit(", ("), 
        col("Item1"),               # Bread
        lit(", "), 
        col("Item2"),               # Coffee
        lit(")")
    ).alias("Output")
)

formatted.show(truncate=False)


+-------------------------+
|Output                   |
+-------------------------+
|evening, (Cake, Coffee)  |
|midday, (Bread, Coffee)  |
|morning, (Coffee, Pastry)|
+-------------------------+



In [45]:
#Question3
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, expr, lag, row_number, when,
    max as max_, min as min_, 
    count, asc, desc
)
from pyspark.sql.window import Window

spark3 = SparkSession.builder.appName("Hw2-Q3").getOrCreate()


In [50]:
from pyspark.sql.functions import expr, col


pop = spark3.read.csv(
    "shared/hw2/populationbycountry19802010millions.csv",
    header=True,
    inferSchema=True
)


current_schema = pop.schema.names
if current_schema[0] == "_c0":
    pop = pop.withColumnRenamed("_c0", "Country")


years = [str(y) for y in range(1980, 2011)]


expr_str = "stack({0}, {1}) as (Year, Population)".format(
    len(years),
    ", ".join([f"'{year}', `{year}`" for year in years])
)

pop_long = pop.select(
    "Country",
    expr(expr_str)
)

pop_long = pop_long.withColumn("Population", col("Population").cast("double"))

pop_long.show(5, truncate=False)
pop_long.printSchema()


+-------------+----+----------+
|Country      |Year|Population|
+-------------+----+----------+
|North America|1980|320.27638 |
|North America|1981|324.44694 |
|North America|1982|328.62014 |
|North America|1983|332.72487 |
|North America|1984|336.72143 |
+-------------+----+----------+
only showing top 5 rows

root
 |-- Country: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Population: double (nullable = true)



25/03/22 03:24:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


In [51]:
regions = ["World","Africa","North America","South America","Asia","Oceania","Europe"]

pop_countries = pop_long.filter(~col("Country").isin(regions))

pop_countries.show(5)


+-------+----+----------+
|Country|Year|Population|
+-------+----+----------+
|Bermuda|1980|   0.05473|
|Bermuda|1981|   0.05491|
|Bermuda|1982|   0.05517|
|Bermuda|1983|   0.05551|
|Bermuda|1984|   0.05585|
+-------+----+----------+
only showing top 5 rows



25/03/22 03:27:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


In [52]:
from pyspark.sql.functions import lag, col

win = Window.partitionBy("Country").orderBy(col("Year").cast("int"))

pop_with_prev = pop_countries \
    .withColumn("PopulationPrev", lag("Population").over(win)) \
    .withColumn("YoYRate", 
        ((col("Population") - col("PopulationPrev")) / col("PopulationPrev") * 100)
    )

pop_after_1980 = pop_with_prev.filter(col("Year") > 1980)

pop_after_1980.show(5)


25/03/22 03:27:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


+-----------+----+----------+--------------+------------------+
|    Country|Year|Population|PopulationPrev|           YoYRate|
+-----------+----+----------+--------------+------------------+
|Afghanistan|1981|  13.67368|       15.0436|-9.106330931425992|
|Afghanistan|1982|  12.57743|      13.67368|-8.017227257036874|
|Afghanistan|1983|  12.43058|      12.57743| -1.16756761913999|
|Afghanistan|1984|  12.75384|      12.43058|2.6005222604254943|
|Afghanistan|1985|  13.09371|      12.75384| 2.664844470371272|
+-----------+----+----------+--------------+------------------+
only showing top 5 rows



In [53]:
from pyspark.sql.functions import row_number

win2 = Window.partitionBy("Year").orderBy(col("YoYRate").desc())

top_country_by_year = pop_after_1980 \
    .withColumn("rank", row_number().over(win2)) \
    .filter(col("rank") == 1) \
    .drop("rank")

top_country_by_year.select("Year", "Country", "Population", "PopulationPrev", "YoYRate") \
    .orderBy("Year") \
    .show(30, truncate=False)


25/03/22 03:27:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


+----+---------------------------------+----------+--------------+------------------+
|Year|Country                          |Population|PopulationPrev|YoYRate           |
+----+---------------------------------+----------+--------------+------------------+
|1981|Western Sahara                   |0.13909   |0.12404       |12.133182844243787|
|1982|Western Sahara                   |0.15455   |0.13909       |11.115105327485802|
|1983|French Guiana                    |0.08      |0.07          |14.285714285714276|
|1984|Qatar                            |0.32447   |0.29241       |10.964057316781224|
|1985|French Guiana                    |0.09      |0.08          |12.499999999999993|
|1986|Qatar                            |0.38726   |0.35603       |8.771732719152874 |
|1987|French Guiana                    |0.1       |0.09          |11.111111111111121|
|1988|Cayman Islands                   |0.0245    |0.02207       |11.010421386497516|
|1989|United Arab Emirates             |1.72206   |1.6

In [54]:
# Max 
top_increase = top_country_by_year.orderBy(col("YoYRate").desc()).limit(1)

#Min
lowest_increase = top_country_by_year.orderBy(col("YoYRate").asc()).limit(1)

print("Year with the highest yoy rate (among yearly champions):")
top_increase.show()

print("Year with the lowest yoy rate (among yearly champions):")
lowest_increase.show()


Year with the highest yoy rate (among yearly champions):


25/03/22 03:28:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


+-------+----+----------+--------------+-----------------+
|Country|Year|Population|PopulationPrev|          YoYRate|
+-------+----+----------+--------------+-----------------+
| Kuwait|1992|   1.41829|       0.95422|48.63343882962002|
+-------+----+----------+--------------+-----------------+

Year with the lowest yoy rate (among yearly champions):


25/03/22 03:28:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
 Schema: _c0, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010
Expected: _c0 but found: 
CSV file: file:///home/jovyan/shared/hw2/populationbycountry19802010millions.csv


+-------+----+----------+--------------+-----------------+
|Country|Year|Population|PopulationPrev|          YoYRate|
+-------+----+----------+--------------+-----------------+
|  Niger|2010|  15.87827|      15.30625|3.737166190281749|
+-------+----+----------+--------------+-----------------+



In [3]:
#Question4
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lower, regexp_replace, split, explode, desc
)

# 创建 SparkSession
spark4 = SparkSession.builder.appName("WordCount").getOrCreate()


In [4]:
lines = spark.read.text("shared/hw2/hw1dir/*")

lines.show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
normalized = lines.select(
    regexp_replace(
        lower(col("value")),   
        "[^0-9a-z]",            
        " "                     
    ).alias("line")
)

normalized.show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:

split_df = normalized.select(split(col("line"), "\\s+").alias("words"))

words_df = split_df.select(explode(col("words")).alias("word"))

words_df = words_df.filter(col("word") != "")

word_count = words_df.groupBy("word").count()


In [7]:
top_20 = word_count.orderBy(col("count").desc())
top_20.show(20, truncate=False)


+----+------+
|word|count |
+----+------+
|the |163547|
|to  |89046 |
|p   |78664 |
|of  |75568 |
|and |72730 |
|in  |56782 |
|a   |53198 |
|for |29770 |
|that|28852 |
|is  |27601 |
|on  |24485 |
|s   |23615 |
|with|19575 |
|are |19417 |
|it  |18231 |
|be  |17998 |
|as  |17796 |
|have|16188 |
|at  |15965 |
|we  |15754 |
+----+------+
only showing top 20 rows



In [16]:
#Question5 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, desc
from pyspark.ml.feature import RegexTokenizer, NGram


spark5 = SparkSession.builder.appName("TrigramAnalysis").getOrCreate()


lines = spark5.read.text("shared/hw2/internet_archive_scifi_v3.txt")

lines = spark5.read \
    .option("encoding", "UTF-8") \
    .text("shared/hw2/internet_archive_scifi_v3.txt")




tokenizer = RegexTokenizer(
    inputCol="value",
    outputCol="words",
    pattern="\\W+",   
    toLowercase=True   
)
tokenized = tokenizer.transform(lines)


tokenized.select("words").show(5, truncate=False)


ngram = NGram(
    n=3,
    inputCol="words",
    outputCol="trigrams"
)
ngramData = ngram.transform(tokenized)


ngramData.select("trigrams").show(5, truncate=False)


trigram_df = ngramData.select(explode(col("trigrams")).alias("trigram"))
trigram_count = trigram_df.groupBy("trigram").count()


trigram_count.orderBy(desc("count")).show(20, truncate=False)


top_10_trigrams = trigram_count.orderBy(desc("count")).limit(10)
top_10_trigrams.show(truncate=False)


spark.stop()




ConnectionRefusedError: [Errno 111] Connection refused